In [65]:
import os

In [122]:
%cd C:\Users\anil\Downloads\faster_rcnn_od\fasterrcnn_sickle_cell_detection

C:\Users\anil\Downloads\faster_rcnn_od\fasterrcnn_sickle_cell_detection


In [67]:
#os.chdir('../')

In [95]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_classes: int
  

In [96]:
from faster_rcnn_od.constants import *
from faster_rcnn_od.utils import read_yaml, create_directories

In [97]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_classes=self.params.CLASSES
            
        )
    
        return prepare_base_model_config

In [98]:
import os
import urllib.request as request
from zipfile import ZipFile


In [99]:
import torchvision
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [117]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
        in_features = self.model.roi_heads.box_predictor.cls_score.in_features
        self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, self.config.params_classes)
       
        
        self.save_model(self.config.base_model_path, self.model)
    

    def save_model(self,path, model):
        model = torch.jit.script(model)
        model.save(path)    



In [118]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

[2023-04-21 18:32:18,650: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2023-04-21 18:32:18,658: INFO: comman: yaml file: params.yaml loaded successfully]
[2023-04-21 18:32:18,661: INFO: comman: created directory at: artifacts]
[2023-04-21 18:32:18,663: INFO: comman: created directory at: artifacts/prepare_base_model]


c:\Users\anil\anaconda3\envs\fasterrcnn\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\anil\anaconda3\envs\fasterrcnn\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [123]:
%pwd

'C:\\Users\\anil\\Downloads\\faster_rcnn_od\\fasterrcnn_sickle_cell_detection'

In [128]:
model_rcnn = torch.jit.load(r'C:\Users\anil\Downloads\faster_rcnn_od\fasterrcnn_sickle_cell_detection\artifacts\prepare_base_model\base_model.pt')
model_rcnn.eval()

RecursiveScriptModule(
  original_name=FasterRCNN
  (transform): RecursiveScriptModule(original_name=GeneralizedRCNNTransform)
  (backbone): RecursiveScriptModule(
    original_name=BackboneWithFPN
    (body): RecursiveScriptModule(
      original_name=IntermediateLayerGetter
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=FrozenBatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
      (layer1): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=Bottleneck
          (conv1): RecursiveScriptModule(original_name=Conv2d)
          (bn1): RecursiveScriptModule(original_name=FrozenBatchNorm2d)
          (conv2): RecursiveScriptModule(original_name=Conv2d)
          (bn2): RecursiveScriptModule(original_name=FrozenBatchNorm2d)
          (conv3): RecursiveScriptModule(original_name=Conv2d)


In [129]:
import pandas as pd
train = pd.read_csv("artifacts\data_ingestion\SIckle_for_rcnn\out.csv")

In [130]:
train

,Unnamed: 0,filename,width,height,class,xmin,ymin,xmax,ymax
0,0,001source_jpg.rf.a1e23e9b54b037f3ba723c3b512c9...,640,640,Normal,217,26,257,93
1,1,001source_jpg.rf.a1e23e9b54b037f3ba723c3b512c9...,640,640,Normal,80,127,117,184
2,2,001source_jpg.rf.a1e23e9b54b037f3ba723c3b512c9...,640,640,Other,279,448,327,500
3,3,001source_jpg.rf.a1e23e9b54b037f3ba723c3b512c9...,640,640,Other,264,134,300,183
4,4,001source_jpg.rf.a1e23e9b54b037f3ba723c3b512c9...,640,640,Other,195,153,240,185
...,...,...,...,...,...,...,...,...,...
27194,27320,source9_jpg.rf.c2e6135ae54fa10f3eaadeb6c848c17...,640,640,Target,289,0,370,79
27195,27321,source9_jpg.rf.c2e6135ae54fa10f3eaadeb6c848c17...,640,640,Target,614,401,640,511
27196,27322,source9_jpg.rf.c2e6135ae54fa10f3eaadeb6c848c17...,640,640,Target,625,552,640,640
27197,27323,source9_jpg.rf.c2e6135ae54fa10f3eaadeb6c848c17...,640,640,Target,519,629,616,639
